In [1]:
import pandas as pd
file_path = 'H1BInfo.csv'
df = pd.read_csv(file_path)

In [2]:
# df.head()

In [3]:
filtered_df = df.iloc[:, [1, 2, 8, 9, 10, 11]]

In [4]:
filtered_df.head()

,Fiscal Year,Employer (Petitioner) Name,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial
0,2023,NaN,0,1,0,0
1,2023,NaN,1,0,0,0
2,2023,NaN,0,0,1,0
3,2023,NaN,0,0,1,0
4,2023,1 800 CONTACTS INC,0,0,2,0


In [5]:
filtered_df_2 = filtered_df.dropna()

In [6]:
row_sums = df.iloc[:,[2,3,4,5]].sum(axis=1)

# Create a boolean mask where True indicates rows with sum >= 1
mask = row_sums >= 1
filtered_df_3 = filtered_df_2[mask]

C:\Users\mayan\AppData\Local\Temp\ipykernel_11716\2556214284.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_sums = df.iloc[:,[2,3,4,5]].sum(axis=1)
C:\Users\mayan\AppData\Local\Temp\ipykernel_11716\2556214284.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_3 = filtered_df_2[mask]


In [7]:
filtered_df_3

,Fiscal Year,Employer (Petitioner) Name,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial
4,2023,1 800 CONTACTS INC,0,0,2,0
5,2023,1 800 FLOWERS COM INC,0,0,5,0
6,2023,1 ADVANCING EDUCATION LLC,0,0,1,0
7,2023,1 BETHESDA DRIVE OPERATING COMPANY LLC DBA ELD...,0,0,1,0
8,2023,1 HOTEL SOUTH BEACH INC,0,0,1,0
...,...,...,...,...,...,...
171968,2021,ZYTUS INC,0,0,3,0
171969,2021,ZYTUS INC,1,0,0,0
171970,2021,ZYWAVE INC,1,0,1,0
171971,2021,ZYWIE INC,0,0,1,0


In [8]:
class CompanyRecord(object):
    
    def __init__(self, company_name):
        self.company_name = company_name
        self.yearly_records_map = {}
    
    def insert(self, record):
        self.yearly_records_map[record.year] = record
    
    def add_records_to_year(self,year,year_record):
        if year in self.yearly_records_map:
            self.yearly_records_map[year].add_record(year_record)
        else:
            self.insert(YearRecord(year,year_record))
        
    def total_H1B_filing_count(self):
        return sum(map(lambda x:x.total_H1B_filing_count_by_year(),self.yearly_records_map.values()))

class YearRecord(object):
    
    def __init__(self, year, sponsor_arr):
        self.year = year
        
        self.initial_approval,self.initial_denial,self.continuing_approval,self.continuing_denial = [int(str(i).replace(",","")) for i in sponsor_arr]
        self.printer()
        
    def add_record(self,sponsor_arr2):
        sponsor_arr2 = [int(str(i).replace(",","")) for i in sponsor_arr2]
        
        self.initial_approval += sponsor_arr2[0]
        self.initial_denial += sponsor_arr2[1]
        self.continuing_approval += sponsor_arr2[2]
        self.continuing_denial += sponsor_arr2[3] 
    
    def printer(self):
        print(f'{self.year} {self.initial_approval} {self.initial_denial} {self.continuing_approval} {self.continuing_denial}')

In [9]:
company_object_map = {}
for row in filtered_df_3.itertuples():
    cr = None
    print(list(row))
    if row[2] not in company_object_map:
        company_object_map[row[2]] = CompanyRecord(row[2])
        cr = company_object_map[row[2]]
        yr = YearRecord(row[1],[row[3],row[4],row[5],row[6]])
        cr.insert(yr)
    else:
        cr = company_object_map[row[2]]
        cr.add_records_to_year(row[1],[row[3],row[4],row[5],row[6]])
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
json_dict = {}


for i in company_object_map:
    company_years = {
        
    }
    for item in company_object_map[i].yearly_records_map.items():
        company_years[item[0]] = item[1].__dict__
    json_dict[i] = company_years

years = set(filtered_df_3.iloc[:,0].unique())

for i in json_dict:
    for j in [2023,2022,2021]:
        try:
            del json_dict[i][j]['year']
        except:
            pass

                

In [11]:
json_dict

{'1 800 CONTACTS INC': {2023: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 2,
   'continuing_denial': 0},
  2022: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 1,
   'continuing_denial': 0},
  2021: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 1,
   'continuing_denial': 0}},
 '1 800 FLOWERS COM INC': {2023: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 5,
   'continuing_denial': 0},
  2022: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 7,
   'continuing_denial': 1},
  2021: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 10,
   'continuing_denial': 0}},
 '1 ADVANCING EDUCATION LLC': {2023: {'initial_approval': 0,
   'initial_denial': 0,
   'continuing_approval': 1,
   'continuing_denial': 0}},
 '1 BETHESDA DRIVE OPERATING COMPANY LLC DBA ELDERWOOD AT HORNELL': {2023: {'initial_approval': 0,
   'initial_denial': 0,
   'co

In [12]:
for i in json_dict:
    for j in years:
        try:
            json_dict[i][j] = [int(json_dict[i][j]['initial_approval']),
                                   int(json_dict[i][j]['initial_denial']),
                                   int(json_dict[i][j]['continuing_approval'])
                                   ,int(json_dict[i][j]['continuing_denial'])]
            
        except:
            pass

In [13]:
json_dict

{'1 800 CONTACTS INC': {2023: [0, 0, 2, 0],
  2022: [0, 0, 1, 0],
  2021: [0, 0, 1, 0]},
 '1 800 FLOWERS COM INC': {2023: [0, 0, 5, 0],
  2022: [0, 0, 7, 1],
  2021: [0, 0, 10, 0]},
 '1 ADVANCING EDUCATION LLC': {2023: [0, 0, 1, 0]},
 '1 BETHESDA DRIVE OPERATING COMPANY LLC DBA ELDERWOOD AT HORNELL': {2023: [0,
   0,
   1,
   0]},
 '1 HOTEL SOUTH BEACH INC': {2023: [0, 0, 1, 0], 2021: [0, 0, 1, 0]},
 '1A1 INNOVATIONS LLC': {2023: [0, 0, 1, 0],
  2022: [1, 0, 0, 0],
  2021: [0, 0, 1, 0]},
 '1CONNECT TECHNOLOGIES INC': {2023: [1, 0, 0, 0]},
 '1HCS FRANCHISING LLC DBA 1HEART CAREGIVER SERVICES': {2023: [1, 0, 0, 0]},
 '1LIFE HEALTHCARE INC': {2023: [1, 0, 2, 0],
  2022: [0, 0, 2, 0],
  2021: [1, 0, 0, 0]},
 '1NTEGER LLC D/B/A KHARON': {2023: [0, 0, 1, 0], 2022: [0, 0, 2, 0]},
 '1POINT3ACRES LLC': {2023: [0, 0, 4, 0], 2022: [1, 0, 0, 0]},
 '1RIVET INTERNATIONAL': {2023: [0, 0, 1, 0]},
 '1SHARPE CAPITAL LLC': {2023: [0, 0, 1, 0], 2021: [0, 0, 1, 0]},
 '1ST AMERICARE LLC': {2023: [0, 1, 0, 0

In [14]:
import json

fp = open("companies.json","w")

json.dump(json_dict,fp)

fp.close()

In [15]:
json_dict['GOOGLE LLC']

{2023: [1264, 6, 4193, 54],
 2022: [1574, 11, 6844, 76],
 2021: [1453, 16, 5001, 51]}